In [ ]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/minsir/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "minsirx.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()

import imaplib
import email
import os
import pandas as pd
from datetime import datetime
import csv
from pandas import DataFrame

In [ ]:
# Function to download attachments with a specific filename from emails


def download_attachments_with_filename(email_id, email_password):
    # Connect to the Outlook IMAP server
    mail = imaplib.IMAP4_SSL("outlook.office365.com", 993)  # Use the Outlook IMAP server and port
    # Login to your Outlook account
    mail.login(email_id, email_password)

    # Select the folder to search for emails
    mail.select("Inbox_1")

    # Search for emails with attachments containing the specified filename
    since_date = datetime(2023, 4, 1)
    since_date_formatted = since_date.strftime('%d-%b-%Y')
    result, data = mail.search(None, '(SINCE {date})'.format(date=since_date_formatted))
    email_nums = data[0].split()
    # email_nums.reverse()

    # Iterate through the list of emails
    for num in email_nums:
        # Fetch the email
        result, data = mail.fetch(num, '(RFC822)')
        raw_email = data[0][1]
        print(type(raw_email))
        if type(raw_email) is int:
            continue
        msg = email.message_from_bytes(raw_email)

        date_str: str = msg.get("Date")
        print("date_str", date_str)
        splitted_1 = date_str.split("+")
        sub_date = splitted_1[0].strip()
        splitted_2 = sub_date.split("-")
        fnal_date = splitted_2[0].strip()
        fnal_date = fnal_date.replace("GMT", "")
        fnal_date = fnal_date.strip()
        frmts = [
            "%a, %d %b %Y %H:%M:%S",
            "%d %b %Y %H:%M:%S"
        ]
        is_dt_made = False
        for a_dt_frmt in frmts:
            try:
                date_obj = datetime.strptime(fnal_date, a_dt_frmt)
            except Exception as err:
                date_obj = None
            if date_obj is not None:
                is_dt_made = True
                break
        if not is_dt_made:
            raise ValueError(f"unable to format date: {fnal_date}")
                

        # Check if the email has attachments
        is_file_found = False
        if msg.get_content_maintype() == 'multipart':
            for part in msg.walk():
                if part.get_content_maintype() == 'multipart':
                    continue
                if part.get('Content-Disposition') is None:
                    continue
                # Save the attachment
                filename = part.get_filename()
                print(filename, date_obj)
                if filename:
                    file_found = False
                    z_plcy_num = None
                    # for plcy_num in policy_mappings:
                    #     if plcy_num in filename:
                    #         z_plcy_num = plcy_num
                    #         file_found = True
                    # if file_found:
                    #     last_date = policy_mappings[z_plcy_num]["received_date"]
                    #     if last_date is None or date_obj > last_date:
                    #         attachment_dir = os.path.join(parent_folder, policy_mappings[z_plcy_num]["folder_name"])
                    #         os.makedirs(str(attachment_dir), exist_ok=True)
                    #         filepath = os.path.join(str(attachment_dir), filename)
                    #         with open(filepath, 'wb') as f:
                    #             f.write(part.get_payload(decode=True))
                    #         policy_mappings[z_plcy_num]["received_date"] = date_obj
                    #         is_file_found = True
                    #     # break
        # if is_file_found:
        #     break

    # Logout from the server
    mail.logout()

# Example usage
email_id = "mirbilal199x@outlook.com"
email_password = "APPollo1947"
download_attachments_with_filename(email_id, email_password)

# Wed, 3 Jan 2024 09:32:34 +0000
# 3 Jan 2024 00:10:59 -0800